In [ ]:
#alejandro briones
#chris kontsis
#sheil patel
#cse471 capstone 

#dev env completed
from google.colab import files
files.upload()

In [ ]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d geomack/spotifyclassification

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('spotifyclassification.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()